In [1]:
# --- CELL 1: Cloaking Utilities (All functions from cloaking_utils.py) ---
# This cell contains all the necessary functions and initializations for cloaking,
# watermarking, detection, and comparison.

import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, LeakyReLU, Add
import os
from PIL import Image, ImageTk
import tkinter as tk # Tkinter is primarily for GUI, not directly used in core utilities logic here, but included for completeness if you paste all
import hashlib # For creating a stable watermark from a secret key

# --- Enhanced GAN Generator Definition ---
def build_generator():
    """
    Builds an improved generator model that creates subtle perturbations.
    Uses residual blocks for better quality and U-Net architecture for preserving details.
    """
    inputs = tf.keras.Input(shape=(None, None, 3))

    # Initial preprocessing
    x = Conv2D(64, 7, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)

    # Downsampling
    x = Conv2D(128, 3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)

    x = Conv2D(256, 3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)

    # Residual blocks
    for _ in range(6):
        res = x
        x = Conv2D(256, 3, padding='same')(x)
        x = BatchNormalization()(x)
        x = LeakyReLU(0.2)(x)
        x = Conv2D(256, 3, padding='same')(x)
        x = BatchNormalization()(x)
        x = Add()([x, res])

    # Upsampling
    x = tf.keras.layers.Conv2DTranspose(128, 3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)

    x = tf.keras.layers.Conv2DTranspose(64, 3, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)

    # Final output with tanh activation (produces perturbations)
    perturbations = Conv2D(3, 7, padding='same', activation='tanh')(x)

    # Scale down perturbations using a Lambda layer (correct way in Keras)
    # The original scale of 0.05 is good for subtle changes.
    perturbations = tf.keras.layers.Lambda(lambda x: x * 0.05)(perturbations)

    # Add perturbations to original image (identity mapping)
    outputs = Add()([inputs, perturbations])

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='generator')
    model.compile(optimizer='adam', loss='mse')
    return model

# Initialize generator once when the module is loaded
generator = build_generator()

# --- Watermarking Configuration ---
WATERMARK_SECRET_KEY = "my_super_secret_cloak_key_123!" # A secret key for the watermark
WATERMARK_STRENGTH = 0.02 # Adjust this for balance between imperceptibility and robustness

def generate_watermark_pattern_dct(size, secret_key):
    """
    Generates a pseudo-random binary pattern for the watermark based on a secret key.
    This pattern is suitable for DCT embedding.
    """
    seed = int(hashlib.sha256(secret_key.encode('utf-8')).hexdigest(), 16) % (2**32 - 1)
    rng = np.random.RandomState(seed)
    watermark_pattern = rng.randint(0, 2, size=size).astype(np.float32)
    return watermark_pattern * 2 - 1

def embed_watermark_dct(image_np_rgb, secret_key=WATERMARK_SECRET_KEY, strength=WATERMARK_STRENGTH):
    """
    Embeds an invisible watermark into the DCT coefficients of an image.
    Works on each color channel separately to embed the watermark.
    """
    img_watermarked = image_np_rgb.copy().astype(np.float32)
    h, w, c = img_watermarked.shape
    watermark_embed_size = (h // 8, w // 8)
    watermark_pattern = generate_watermark_pattern_dct(watermark_embed_size, secret_key)

    for i in range(c):
        img_channel = img_watermarked[:, :, i]
        dct_channel = cv2.dct(img_channel)
        embed_row_start = watermark_embed_size[0] // 2 
        embed_col_start = watermark_embed_size[1] // 2 
        block_rows = min(watermark_embed_size[0], h - embed_row_start)
        block_cols = min(watermark_embed_size[1], w - embed_col_start)
        dct_channel[embed_row_start : embed_row_start + block_rows,
                    embed_col_start : embed_col_start + block_cols] += strength * watermark_pattern[:block_rows, :block_cols]
        img_watermarked[:, :, i] = cv2.idct(dct_channel)

    img_watermarked = np.clip(img_watermarked, 0, 255).astype(np.uint8)
    return img_watermarked

def detect_watermark_dct(image_np_rgb, secret_key=WATERMARK_SECRET_KEY, threshold_correlation=0.01):
    """
    Detects the presence of an embedded watermark in the DCT coefficients of an image.
    """
    img = image_np_rgb.copy().astype(np.float32)
    h, w, c = img.shape
    watermark_embed_size = (h // 8, w // 8)
    expected_watermark_pattern = generate_watermark_pattern_dct(watermark_embed_size, secret_key)
    correlations = []

    for i in range(c):
        img_channel = img[:, :, i]
        dct_channel = cv2.dct(img_channel)
        embed_row_start = watermark_embed_size[0] // 2
        embed_col_start = watermark_embed_size[1] // 2
        block_rows = min(watermark_embed_size[0], h - embed_row_start)
        block_cols = min(watermark_embed_size[1], w - embed_col_start)
        extracted_block = dct_channel[embed_row_start : embed_row_start + block_rows,
                                      embed_col_start : embed_col_start + block_cols]

        if extracted_block.shape[0] == 0 or extracted_block.shape[1] == 0 or np.std(extracted_block) < 1e-6:
            correlations.append(0)
            continue

        current_expected_pattern = expected_watermark_pattern[:extracted_block.shape[0], :extracted_block.shape[1]]
        
        if np.std(current_expected_pattern) > 1e-6:
            correlation = np.corrcoef(extracted_block.flatten(), current_expected_pattern.flatten())[0, 1]
            correlations.append(correlation)
        else:
            correlations.append(0)

    avg_correlation = np.mean(correlations)
    print(f"Watermark Detection: Average correlation = {avg_correlation:.4f}, Threshold = {threshold_correlation:.4f}")
    return avg_correlation > threshold_correlation

def cloak_image(image_path, output_dir=None):
    """
    Applies subtle GAN-based perturbations to an image and then embeds a robust watermark.
    """
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError(f"Could not read image from {image_path}")

    if len(img.shape) == 2:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    elif img.shape[2] == 4:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
    else:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    img_normalized = img_rgb.astype(np.float32) / 127.5 - 1.0
    input_tensor = np.expand_dims(img_normalized, axis=0)
    
    try:
        cloaked_img_normalized = generator.predict(input_tensor, verbose=0)[0]
    except Exception as e:
        raise ValueError(f"GAN prediction failed: {str(e)}. Check TensorFlow/GPU setup.")

    cloaked_img_normalized = np.clip(cloaked_img_normalized, -1, 1)
    cloaked_img_pre_watermark = ((cloaked_img_normalized + 1) * 127.5).astype(np.uint8)
    cloaked_img_final = embed_watermark_dct(cloaked_img_pre_watermark)

    cloaked_output_path = None
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        base_name = os.path.basename(image_path)
        name, ext = os.path.splitext(base_name)
        cloaked_output_path = os.path.join(output_dir, f"{name}_cloaked_wm.png")
        cloaked_bgr = cv2.cvtColor(cloaked_img_final, cv2.COLOR_RGB2BGR)
        cv2.imwrite(cloaked_output_path, cloaked_bgr)

    return img_rgb, cloaked_img_final, cloaked_output_path

def is_cloaked(image_path, adaptive_threshold=True):
    """
    Determines if an image has been processed by the cloaking utility.
    """
    try:
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError(f"Invalid image file: {image_path}")
        
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        watermark_detected = detect_watermark_dct(img_rgb)
        
        laplacian = cv2.Laplacian(img_gray, cv2.CV_64F)
        noise_energy = np.var(laplacian)

        dft = np.fft.fft2(img_gray)
        dft_shift = np.fft.fftshift(dft)
        magnitude_spectrum = 20 * np.log(np.abs(dft_shift) + 1e-10)
        
        rows, cols = img_gray.shape
        crow, ccol = rows // 2, cols // 2
        mask = np.zeros((rows, cols), np.uint8)
        r_outer = min(crow, ccol) * 0.4
        r_inner = min(crow, ccol) * 0.1
        cv2.circle(mask, (ccol, crow), int(r_outer), 1, -1)
        cv2.circle(mask, (ccol, crow), int(r_inner), 0, -1)
        
        high_freq_spectrum = magnitude_spectrum * mask
        high_freq_energy = np.sum(high_freq_spectrum) / (np.sum(mask) + 1e-10)
        
        if adaptive_threshold:
            avg_brightness = np.mean(img_gray)
            threshold = 10 + (avg_brightness / 15.0)
        else:
            threshold = 15.0
        
        cloaked_by_pattern = noise_energy > threshold and high_freq_energy > 70
        
        print(f"Detection Metrics - Noise: {noise_energy:.2f}, Freq: {high_freq_energy:.2f}, Threshold: {threshold:.2f}, Watermark: {watermark_detected}")
        
        return cloaked_by_pattern or watermark_detected

    except Exception as e:
        print(f"Detection error: {str(e)}")
        return False

def compare_images(orig_path, cloaked_path, output_dir=None):
    """
    Generates a visual comparison showing significant differences between two images,
    marked in green.
    """
    orig = cv2.imread(orig_path)
    cloaked = cv2.imread(cloaked_path)

    if orig is None or cloaked is None:
        raise ValueError("Could not read one or both images for comparison.")

    if orig.shape != cloaked.shape:
        cloaked = cv2.resize(cloaked, (orig.shape[1], orig.shape[0]))

    diff = cv2.absdiff(orig, cloaked)
    diff_gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    thresh = cv2.adaptiveThreshold(diff_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    kernel = np.ones((3,3), np.uint8)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    marked = orig.copy()
    marked[thresh == 255] = [0, 255, 0]

    comparison_output_path = None
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)
        base_name = os.path.basename(orig_path)
        name, ext = os.path.splitext(base_name)
        comparison_output_path = os.path.join(output_dir, f"{name}_comparison{ext}")
        cv2.imwrite(comparison_output_path, marked)
        
    return cv2.cvtColor(marked, cv2.COLOR_BGR2RGB), comparison_output_path

def show_image_on_canvas(canvas, image_path, max_width=600, max_height=400):
    """
    Loads an image from path, resizes it to fit within max_width/height,
    and displays it on the given Tkinter canvas.
    """
    try:
        img_pil = Image.open(image_path)
        img_pil.thumbnail((max_width, max_height), Image.Resampling.LANCZOS)
        photo = ImageTk.PhotoImage(img_pil)
        canvas.delete("all")
        canvas.config(width=photo.width(), height=photo.height())
        canvas.create_image(photo.width() / 2, photo.height() / 2, image=photo)
        return photo
    except Exception as e:
        print(f"Error displaying image on canvas: {e}")
        return None

def show_numpy_image_on_canvas(canvas, img_np_rgb, max_width=600, max_height=400):
    """
    Displays a NumPy array image (RGB) on the given Tkinter canvas.
    """
    try:
        img_pil = Image.fromarray(img_np_rgb)
        img_pil.thumbnail((max_width, max_height), Image.Resampling.LANCZOS)
        photo = ImageTk.PhotoImage(img_pil)
        canvas.delete("all")
        canvas.config(width=photo.width(), height=photo.height())
        canvas.create_image(photo.width() / 2, photo.height() / 2, image=photo)
        return photo
    except Exception as e:
        print(f"Error displaying numpy image on canvas: {e}")
        return None


# --- CELL 2: Tkinter Application (from your cloak-image-app-fixed-v2) ---
# This cell contains your GUI application.

class CloakImageApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Cloak Image - AI Training Disruptor")
        self.root.geometry("700x550")
        self.root.resizable(False, False)
        
        self.output_dir = os.path.join(os.path.expanduser("~"), "CloakedImages")
        self.original_image_path = ""
        self.cloaked_image_path = ""
        self.original_photo = None
        self.cloaked_photo = None
        
        self.create_widgets()
        os.makedirs(self.output_dir, exist_ok=True)

    def create_widgets(self):
        main_frame = tk.Frame(self.root, padx=15, pady=15)
        main_frame.pack(expand=True, fill=tk.BOTH)

        tk.Label(main_frame, text="AI Image Cloaker", font=("Arial", 16, "bold")).pack(pady=10)
        tk.Label(main_frame, text=f"Select an image to apply AI-disrupting cloaking.\nCloaked images saved to: {self.output_dir}", 
                 wraplength=600).pack(pady=5)

        button_frame = tk.Frame(main_frame)
        button_frame.pack(pady=20)
        
        tk.Button(button_frame, text="Select Image", command=self.select_image_for_cloaking,
                  font=("Arial", 10, "bold"), bg="#4CAF50", fg="white", relief=tk.RAISED, bd=3).pack(side=tk.LEFT, padx=10)
        
        self.cloak_button = tk.Button(button_frame, text="Cloak & Save", command=self.perform_cloaking,
                                       font=("Arial", 10, "bold"), bg="#2196F3", fg="white", relief=tk.RAISED, bd=3, state=tk.DISABLED)
        self.cloak_button.pack(side=tk.LEFT, padx=10)

        display_frame = tk.Frame(main_frame, bd=2, relief=tk.GROOVE)
        display_frame.pack(expand=True, fill=tk.BOTH, pady=10)

        original_frame = tk.Frame(display_frame)
        original_frame.pack(side=tk.LEFT, expand=True, fill=tk.BOTH, padx=5, pady=5)
        tk.Label(original_frame, text="Original Image", font=("Arial", 10, "italic")).pack()
        self.original_canvas = tk.Canvas(original_frame, bg="lightgray", width=300, height=300, relief=tk.SUNKEN, bd=1)
        self.original_canvas.pack(expand=True, fill=tk.BOTH)

        cloaked_frame = tk.Frame(display_frame)
        cloaked_frame.pack(side=tk.RIGHT, expand=True, fill=tk.BOTH, padx=5, pady=5)
        tk.Label(cloaked_frame, text="Cloaked Image", font=("Arial", 10, "italic")).pack()
        self.cloaked_canvas = tk.Canvas(cloaked_frame, bg="lightgray", width=300, height=300, relief=tk.SUNKEN, bd=1)
        self.cloaked_canvas.pack(expand=True, fill=tk.BOTH)

        self.status_label = tk.Label(main_frame, text="Ready: Select an image to begin.", bd=1, relief=tk.SUNKEN, anchor=tk.W)
        self.status_label.pack(side=tk.BOTTOM, fill=tk.X, pady=5)

    def update_status(self, message):
        self.status_label.config(text=message)
        self.root.update_idletasks()

    def select_image_for_cloaking(self):
        file_types = [
            ("Image Files", "*.jpg *.jpeg *.png *.bmp"),
            ("JPEG Files", "*.jpg *.jpeg"),
            ("PNG Files", "*.png"),
            ("All Files", "*.*")
        ]
        
        file_path = filedialog.askopenfilename(title="Select Image to Cloak", filetypes=file_types)
        
        if file_path:
            self.original_image_path = file_path
            self.cloaked_image_path = ""
            self.update_status(f"Selected: {os.path.basename(file_path)}")
            
            self.original_canvas.delete("all")
            self.cloaked_canvas.delete("all")
            
            try:
                img = Image.open(file_path)
                img.thumbnail((300, 300), Image.Resampling.LANCZOS)
                self.original_photo = ImageTk.PhotoImage(img)
                self.original_canvas.create_image(150, 150, anchor=tk.CENTER, image=self.original_photo)
                self.cloak_button.config(state=tk.NORMAL)
            except Exception as e:
                messagebox.showerror("Error", f"Failed to load image: {str(e)}")
                self.update_status("Error loading image.")
                self.cloak_button.config(state=tk.DISABLED)
        else:
            self.update_status("Image selection cancelled.")

    def perform_cloaking(self):
        if not self.original_image_path:
            messagebox.showwarning("No Image Selected", "Please select an image first.")
            return

        try:
            self.update_status("Processing: Applying cloaking perturbations...")
            self.cloak_button.config(state=tk.DISABLED)
            self.root.update_idletasks()

            # Call the cloak_image function from the utility definitions above
            _, cloaked_img_np_rgb, cloaked_path = cloak_image(self.original_image_path, self.output_dir)

            if cloaked_path and os.path.exists(cloaked_path):
                self.cloaked_image_path = cloaked_path
                img = Image.open(cloaked_path)
                img.thumbnail((300, 300), Image.Resampling.LANCZOS)
                self.cloaked_photo = ImageTk.PhotoImage(img)
                self.cloaked_canvas.create_image(150, 150, anchor=tk.CENTER, image=self.cloaked_photo)
                
                self.update_status(f"Cloaking complete! Saved to: {os.path.basename(cloaked_path)}")
                messagebox.showinfo("Success", f"Image cloaked successfully!\nSaved to:\n{cloaked_path}")
            else:
                messagebox.showerror("Error", "Cloaking failed to save the image. Check console for details if any.")
                self.update_status("Cloaking failed. See console for errors.")
                
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred during cloaking: {str(e)}\n\n"
                                          "Please check your Python console/terminal for a detailed traceback.")
            self.update_status(f"Error: {str(e)}. Check console.")
        finally:
            self.cloak_button.config(state=tk.NORMAL)

# --- CELL 3: AI Testing Script ---
# This cell contains the script to evaluate cloaking effectiveness against a pre-trained AI.

from tensorflow.keras.applications.resnet import ResNet50, preprocess_input, decode_predictions
# Note: cloak_image and is_cloaked are defined in CELL 1

def run_ai_disruption_test():
    """
    Runs a test to evaluate how effectively cloaked images disrupt a pre-trained AI model.
    """
    TEST_IMAGES_DIR = 'test_images' # Ensure this folder exists and has images
    CLOAKED_OUTPUT_DIR = 'test_output/cloaked'
    COMPARISON_OUTPUT_DIR = 'test_output/comparison'

    os.makedirs(CLOAKED_OUTPUT_DIR, exist_ok=True)
    os.makedirs(COMPARISON_OUTPUT_DIR, exist_ok=True)

    print("Loading pre-trained ResNet50 model (this may take a moment)...")
    try:
        model = ResNet50(weights='imagenet')
        print("ResNet50 model loaded successfully.")
    except Exception as e:
        print(f"Error loading ResNet50 model: {e}")
        print("Please ensure you have an active internet connection or try installing TensorFlow/Keras dependencies properly.")
        return # Exit the function if model can't be loaded

    print("\n--- Starting AI Disruption Test ---")
    print(f"Testing images from: {TEST_IMAGES_DIR}")
    print(f"Cloaked outputs saved to: {CLOAKED_OUTPUT_DIR}")

    total_images_tested = 0
    successfully_disrupted = 0
    watermark_detected_on_cloaked = 0

    image_files = [f for f in os.listdir(TEST_IMAGES_DIR) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))]

    if not image_files:
        print(f"No image files found in {TEST_IMAGES_DIR}. Please add some images to test.")
        return

    for filename in image_files:
        original_image_path = os.path.join(TEST_IMAGES_DIR, filename)
        print(f"\n--- Processing image: {filename} ---")

        total_images_tested += 1

        # --- Step 1: Cloak the image using your utility ---
        try:
            original_img_np, cloaked_img_np, saved_cloaked_path = cloak_image(original_image_path, CLOAKED_OUTPUT_DIR)
            if not saved_cloaked_path:
                print(f"Skipping {filename}: Cloaking failed to save output.")
                continue
        except Exception as e:
            print(f"Error cloaking {filename}: {e}. Skipping this image.")
            continue

        # --- Step 2: Prepare images for ResNet50 model ---
        img_original_resized = cv2.resize(original_img_np, (224, 224))
        img_cloaked_resized = cv2.resize(cloaked_img_np, (224, 224))

        input_original = preprocess_input(np.expand_dims(img_original_resized, axis=0))
        input_cloaked = preprocess_input(np.expand_dims(img_cloaked_resized, axis=0))

        # --- Step 3: Get predictions from the pre-trained AI model ---
        print("Getting AI predictions...")
        preds_original = model.predict(input_original, verbose=0)
        preds_cloaked = model.predict(input_cloaked, verbose=0)

        top_original_label = decode_predictions(preds_original, top=1)[0][0][1]
        top_cloaked_label = decode_predictions(preds_cloaked, top=1)[0][0][1]

        print(f"Original AI prediction: {top_original_label}")
        print(f"Cloaked AI prediction:  {top_cloaked_label}")

        # --- Step 4: Evaluate AI Disruption ---
        if top_original_label != top_cloaked_label:
            print(f"RESULT: SUCCESS! Cloaking caused AI to misclassify from '{top_original_label}' to '{top_cloaked_label}'.")
            successfully_disrupted += 1
        else:
            print(f"RESULT: Cloaking DID NOT disrupt AI. Still classified as '{top_original_label}'.")

        # --- Step 5: Check your 'is_cloaked' utility on the cloaked image ---
        try:
            is_it_cloaked_by_util = is_cloaked(saved_cloaked_path)
            print(f"Your 'is_cloaked' utility says it's cloaked: {is_it_cloaked_by_util}")
            if is_it_cloaked_by_util:
                watermark_detected_on_cloaked += 1
        except Exception as e:
            print(f"Error calling is_cloaked utility: {e}")
        
        # --- Step 6: (Optional) Test the screenshot loophole using is_cloaked ---
        screenshot_test_path = os.path.join(CLOAKED_OUTPUT_DIR, f"screenshot_of_{filename.split('.')[0]}_cloaked_wm.png")
        if os.path.exists(screenshot_test_path):
            print(f"--- Testing 'is_cloaked' on a simulated screenshot ({os.path.basename(screenshot_test_path)}) ---")
            is_screenshot_cloaked_by_util = is_cloaked(screenshot_test_path)
            print(f"Your 'is_cloaked' utility says screenshot is cloaked: {is_screenshot_cloaked_by_util}")
        else:
            print(f"To test screenshot robustness, manually take a screenshot of '{os.path.basename(saved_cloaked_path)}'")
            print(f"and save it as '{os.path.basename(screenshot_test_path)}' then re-run the script.")


    # --- Test Summary ---
    print("\n--- AI Disruption Test Summary ---")
    print(f"Total images tested: {total_images_tested}")
    print(f"Images where AI classification was disrupted: {successfully_disrupted}")
    if total_images_tested > 0:
        disruption_rate = (successfully_disrupted / total_images_tested) * 100
        print(f"AI Disruption Rate (ASR): {disruption_rate:.2f}%")
        print(f"Your 'is_cloaked' utility detected cloak/watermark on {watermark_detected_on_cloaked} out of {total_images_tested} directly cloaked images.")
    else:
        print("No images were successfully processed for testing.")


# --- CELL 4: Execution Blocks (Run these separately based on what you want to do) ---

# To run the Tkinter GUI:
# if __name__ == "__main__": # In Jupyter, this check often behaves differently, so directly call if needed.
#     root = tk.Tk()
#     app = CloakImageApp(root)
#     root.mainloop()

# To run the AI Disruption Test (after creating 'test_images' folder with images):
# run_ai_disruption_test()


This function below Shields the image using small perturbation or pixel deflation to confuse the AI model.


In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox, simpledialog
import os
import numpy as np
from PIL import Image, ImageTk, ImageDraw, ImageFont

class CloakImageApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Cloak Image - AI Training Disruptor")
        self.root.geometry("800x600")  # Increased size for watermark controls
        self.root.resizable(False, False)

        # Watermark settings
        self.watermark_text = "CONFIDENTIAL"
        self.watermark_font = "arial.ttf"
        self.watermark_size = 20
        self.watermark_color = (255, 255, 255, 128)  # White with transparency
        self.watermark_position = "bottom-right"  # Default position

        # Set the custom output directory
        self.output_dir = r"C:\Users\Asus\OneDrive\Pictures\Cloaking"
        
        self.original_image_path = ""
        self.cloaked_image_path = ""
        self.original_photo = None
        self.cloaked_photo = None

        self.create_widgets()

    def create_widgets(self):
        main_frame = tk.Frame(self.root, padx=15, pady=15)
        main_frame.pack(expand=True, fill=tk.BOTH)

        tk.Label(main_frame, text="AI Image Cloaker", font=("Arial", 16, "bold")).pack(pady=10)
        tk.Label(main_frame, 
                text=f"Select an image to apply AI-disrupting cloaking. Cloaked images will be saved to:\n{self.output_dir}", 
                wraplength=600).pack(pady=5)

        # Watermark controls frame
        watermark_frame = tk.Frame(main_frame)
        watermark_frame.pack(fill=tk.X, pady=5)
        
        tk.Button(watermark_frame, text="Set Watermark Text", command=self.set_watermark_text,
                 font=("Arial", 9), bg="#E0E0E0").pack(side=tk.LEFT, padx=5)
        
        tk.Button(watermark_frame, text="Set Watermark Size", command=self.set_watermark_size,
                 font=("Arial", 9), bg="#E0E0E0").pack(side=tk.LEFT, padx=5)
        
        tk.Button(watermark_frame, text="Set Watermark Color", command=self.set_watermark_color,
                 font=("Arial", 9), bg="#E0E0E0").pack(side=tk.LEFT, padx=5)
        
        tk.Button(watermark_frame, text="Set Position", command=self.set_watermark_position,
                 font=("Arial", 9), bg="#E0E0E0").pack(side=tk.LEFT, padx=5)

        # Buttons frame
        button_frame = tk.Frame(main_frame)
        button_frame.pack(pady=10)
        
        tk.Button(button_frame, text="Select Image", command=self.select_image_for_cloaking,
                 font=("Arial", 10, "bold"), bg="#4CAF50", fg="white",
                 relief=tk.RAISED, bd=3).pack(side=tk.LEFT, padx=10)
        
        self.cloak_button = tk.Button(button_frame, text="Cloak & Save", command=self.perform_cloaking,
                                     font=("Arial", 10, "bold"), bg="#2196F3", fg="white",
                                     relief=tk.RAISED, bd=3, state=tk.DISABLED)
        self.cloak_button.pack(side=tk.LEFT, padx=10)

        # Image display areas
        display_frame = tk.Frame(main_frame, bd=2, relief=tk.GROOVE)
        display_frame.pack(expand=True, fill=tk.BOTH, pady=10)

        # Original Image
        original_frame = tk.Frame(display_frame)
        original_frame.pack(side=tk.LEFT, expand=True, fill=tk.BOTH, padx=5, pady=5)
        tk.Label(original_frame, text="Original Image", font=("Arial", 10, "italic")).pack()
        self.original_canvas = tk.Canvas(original_frame, bg='lightgray', width=350, height=350, relief=tk.SUNKEN, bd=1)
        self.original_canvas.pack(expand=True, fill=tk.BOTH)

        # Cloaked Image
        cloaked_frame = tk.Frame(display_frame)
        cloaked_frame.pack(side=tk.RIGHT, expand=True, fill=tk.BOTH, padx=5, pady=5)
        tk.Label(cloaked_frame, text="Cloaked Image", font=("Arial", 10, "italic")).pack()
        self.cloaked_canvas = tk.Canvas(cloaked_frame, bg='lightgray', width=350, height=350, relief=tk.SUNKEN, bd=1)
        self.cloaked_canvas.pack(expand=True, fill=tk.BOTH)

        # Status Label
        self.status_label = tk.Label(main_frame, text="Ready: Select an image to begin.", 
                                   bd=1, relief=tk.SUNKEN, anchor=tk.W)
        self.status_label.pack(side=tk.BOTTOM, fill=tk.X, pady=5)

    def set_watermark_text(self):
        new_text = simpledialog.askstring("Watermark Text", "Enter watermark text:", 
                                         initialvalue=self.watermark_text)
        if new_text:
            self.watermark_text = new_text
            self.update_status(f"Watermark text set to: {self.watermark_text}")
            if self.cloaked_image_path:  # Refresh display if we have a cloaked image
                self.display_cloaked_image(self.cloaked_image_path)

    def set_watermark_size(self):
        new_size = simpledialog.askinteger("Watermark Size", "Enter font size (8-72):", 
                                          initialvalue=self.watermark_size,
                                          minvalue=8, maxvalue=72)
        if new_size:
            self.watermark_size = new_size
            self.update_status(f"Watermark size set to: {self.watermark_size}")
            if self.cloaked_image_path:
                self.display_cloaked_image(self.cloaked_image_path)

    def set_watermark_color(self):
        color = tk.colorchooser.askcolor(title="Choose Watermark Color", 
                                        initialcolor=self.watermark_color[:3])[0]
        if color:
            self.watermark_color = (int(color[0]), int(color[1]), int(color[2]), 128)
            self.update_status(f"Watermark color set to: {color}")
            if self.cloaked_image_path:
                self.display_cloaked_image(self.cloaked_image_path)
    
    def set_watermark_position(self):
        position = simpledialog.askstring("Watermark Position", 
                                         "Enter position (top-left, top-right, bottom-left, bottom-right):",
                                         initialvalue=self.watermark_position)
        if position and position.lower() in ["top-left", "top-right", "bottom-left", "bottom-right"]:
            self.watermark_position = position.lower()
            self.update_status(f"Watermark position set to: {self.watermark_position}")
            if self.cloaked_image_path:
                self.display_cloaked_image(self.cloaked_image_path)

    def update_status(self, message):
        self.status_label.config(text=message)
        self.root.update_idletasks()

    def select_image_for_cloaking(self):
        file_path = filedialog.askopenfilename(
            title="Select Image to Cloak",
            filetypes=[("Image Files", "*.jpg *.jpeg *.png *.bmp")]
        )
        if file_path:
            self.original_image_path = file_path
            self.cloaked_image_path = ""
            self.update_status(f"Selected: {os.path.basename(file_path)}")
            
            self.original_canvas.delete("all")
            self.cloaked_canvas.delete("all")
            
            try:
                img = Image.open(file_path)
                img.thumbnail((350, 350))
                self.original_photo = ImageTk.PhotoImage(img)
                self.original_canvas.create_image(
                    175, 175, anchor=tk.CENTER, image=self.original_photo
                )
                self.cloak_button.config(state=tk.NORMAL)
            except Exception as e:
                messagebox.showerror("Error", f"Failed to load image: {str(e)}")
                self.update_status("Error loading image.")
        else:
            self.update_status("Image selection cancelled.")

    def add_watermark(self, image):
        """Adds watermark to the image in the specified position"""
        try:
            # Make sure the image is in RGBA mode
            if image.mode != 'RGBA':
                image = image.convert('RGBA')
            
            # Create a transparent layer for the watermark
            watermark = Image.new('RGBA', image.size, (0, 0, 0, 0))
            draw = ImageDraw.Draw(watermark)
            
            # Use a default font if the specified one isn't available
            try:
                font = ImageFont.truetype(self.watermark_font, self.watermark_size)
            except:
                font = ImageFont.load_default()
            
            # Calculate text size
            text_bbox = draw.textbbox((0, 0), self.watermark_text, font=font)
            text_width = text_bbox[2] - text_bbox[0]
            text_height = text_bbox[3] - text_bbox[1]
            margin = 20  # 20px margin from edges
            
            # Calculate position based on selected option
            if self.watermark_position == "bottom-right":
                x = image.width - text_width - margin
                y = image.height - text_height - margin
            elif self.watermark_position == "top-right":
                x = image.width - text_width - margin
                y = margin
            elif self.watermark_position == "bottom-left":
                x = margin
                y = image.height - text_height - margin
            else:  # top-left
                x = margin
                y = margin
            
            # Draw the text
            draw.text((x, y), self.watermark_text, font=font, fill=self.watermark_color)
            
            # Combine with original image
            watermarked = Image.alpha_composite(image, watermark)
            
            return watermarked
            
        except Exception as e:
            print(f"Error adding watermark: {e}")
            return image  # Return original if watermark fails

    def display_cloaked_image(self, image_path):
        """Display the cloaked image with watermark on canvas"""
        try:
            img = Image.open(image_path)
            img.thumbnail((350, 350))
            self.cloaked_photo = ImageTk.PhotoImage(img)
            self.cloaked_canvas.create_image(
                175, 175, anchor=tk.CENTER, image=self.cloaked_photo
            )
        except Exception as e:
            messagebox.showerror("Error", f"Failed to display cloaked image: {str(e)}")

    def cloak_image(self, image_path):
        """Apply cloaking perturbations and watermark to an image"""
        try:
            # Create output directory if it doesn't exist
            os.makedirs(self.output_dir, exist_ok=True)
            
            # Load the image
            img = Image.open(image_path)
            
            # Convert to numpy array for processing
            img_array = np.array(img)
            
            # Convert to float for calculations
            img_float = img_array.astype(np.float32) / 255.0
            
            # Generate random perturbations
            perturbation = np.random.normal(loc=0, scale=0.05, size=img_float.shape)
            
            # Apply perturbations
            cloaked_img = img_float + perturbation
            cloaked_img = np.clip(cloaked_img, 0, 1)  # Ensure valid pixel values
            
            # Convert back to PIL Image
            cloaked_img = (cloaked_img * 255).astype(np.uint8)
            cloaked_img = Image.fromarray(cloaked_img)
            
            # Add watermark
            watermarked_img = self.add_watermark(cloaked_img)
            
            # Save the final image
            base_name = os.path.basename(image_path)
            name, ext = os.path.splitext(base_name)
            output_path = os.path.join(self.output_dir, f"{name}_cloaked{ext}")
            
            # Convert to RGB if saving as JPEG
            if ext.lower() in ('.jpg', '.jpeg'):
                watermarked_img = watermarked_img.convert('RGB')
            
            watermarked_img.save(output_path)
            return output_path
            
        except Exception as e:
            raise Exception(f"Cloaking failed: {str(e)}")

    def perform_cloaking(self):
        if not self.original_image_path:
            messagebox.showwarning("No Image Selected", "Please select an image first.")
            return

        try:
            self.update_status("Processing: Applying cloaking perturbations and watermark...")
            self.cloak_button.config(state=tk.DISABLED)
            self.root.update()
            
            # Call our cloaking function
            cloaked_path = self.cloak_image(self.original_image_path)
            
            if cloaked_path and os.path.exists(cloaked_path):
                self.cloaked_image_path = cloaked_path
                self.display_cloaked_image(cloaked_path)
                
                self.update_status(f"Cloaking complete! Saved to: {cloaked_path}")
                messagebox.showinfo(
                    "Success",
                    f"Image cloaked and watermarked successfully!\nSaved to:\n{cloaked_path}"
                )
            else:
                messagebox.showerror("Error", "Cloaking failed to save the image.")
                self.update_status("Cloaking failed.")
                
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred during cloaking: {str(e)}")
            self.update_status(f"Error: {str(e)}")
        finally:
            self.cloak_button.config(state=tk.NORMAL)

if __name__ == "__main__":
    root = tk.Tk()
    app = CloakImageApp(root)
    root.mainloop()

The Function below tell wether the image is cloaked or not .


In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox
import os
from PIL import Image, ImageTk, ImageStat # Pillow library for image processing

class DetectCloakingApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Detect Cloaking - AI Training Disruptor")
        self.root.geometry("600x450")
        self.root.resizable(False, False)

        self.image_to_check_path = ""
        self.displayed_photo = None # To hold the PhotoImage object

        self.create_widgets()

    def create_widgets(self):
        # Main frame for padding
        main_frame = tk.Frame(self.root, padx=15, pady=15)
        main_frame.pack(expand=True, fill=tk.BOTH)

        # Title and description
        tk.Label(main_frame, text="AI Cloaking Detector", font=("Arial", 16, "bold")).pack(pady=10)
        tk.Label(main_frame, text="Select an image to check if it has been cloaked.", wraplength=400).pack(pady=5)

        # Button frame for organization
        button_frame = tk.Frame(main_frame)
        button_frame.pack(pady=20)

        # Select Image Button
        tk.Button(button_frame, text="Select Image to Check", command=self.select_image_for_detection,
                  font=("Arial", 10, "bold"), bg="#607D8B", fg="white",
                  relief=tk.RAISED, bd=3).pack(side=tk.LEFT, padx=10)

        # Analyze Image Button (initially disabled)
        self.detect_button = tk.Button(button_frame, text="Analyze Image", command=self.perform_detection,
                                       font=("Arial", 10, "bold"), bg="#FFC107", fg="black",
                                       relief=tk.RAISED, bd=3, state=tk.DISABLED)
        self.detect_button.pack(side=tk.LEFT, padx=10)

        # Canvas to display the image
        self.image_canvas = tk.Canvas(main_frame, bg='lightgray', width=500, height=300, relief=tk.SUNKEN, bd=1)
        self.image_canvas.pack(expand=True, fill=tk.BOTH, pady=10)

        # Status bar label
        self.status_label = tk.Label(main_frame, text="Ready: Select an image to analyze.", bd=1, relief=tk.SUNKEN, anchor=tk.W)
        self.status_label.pack(side=tk.BOTTOM, fill=tk.X, pady=5)

    def update_status(self, message):
        """Updates the text in the status bar."""
        self.status_label.config(text=message)
        self.root.update_idletasks() # Ensure the GUI updates immediately

    def select_image_for_detection(self):
        """Opens a file dialog to select an image and displays it."""
        file_path = filedialog.askopenfilename(
            title="Select Image to Detect Cloaking",
            filetypes=[("Image Files", "*.jpg *.jpeg *.png *.bmp")]
        )
        if file_path:
            self.image_to_check_path = file_path
            self.update_status(f"Selected: {os.path.basename(file_path)}")
            
            # Ensure canvas dimensions are available before drawing
            self.root.update_idletasks()
            self.displayed_photo = self._show_image_on_canvas(self.image_canvas, file_path)
            self.detect_button.config(state=tk.NORMAL) # Enable the Analyze button
        else:
            self.update_status("Image selection cancelled.")
            self.detect_button.config(state=tk.DISABLED) # Disable if no image is selected

    def perform_detection(self):
        """Performs the cloaking detection using the internal _is_cloaked function."""
        if not self.image_to_check_path:
            messagebox.showwarning("No Image Selected", "Please select an image first.")
            return

        try:
            self.update_status("Analyzing: Checking for cloaking signatures...")
            
            # Call the internal cloaking detection function
            is_cloaked_result = self._is_cloaked(self.image_to_check_path)

            status_text = "CLOAKED" if is_cloaked_result else "NOT CLOAKED"
            
            # Show a messagebox with the result
            messagebox.showinfo("Detection Result",
                                f"Image Analysis Complete:\n\n"
                                f"Status: {status_text}\n\n"
                                f"This detection uses advanced metrics to identify\n"
                                f"GAN-based perturbations designed to disrupt AI training.")
            
            self.update_status(f"Detection complete: Image is {status_text}.")

        except Exception as e:
            # Handle any errors during detection
            messagebox.showerror("Error", f"An error occurred during detection: {str(e)}")
            self.update_status("Error during detection.")

    def _show_image_on_canvas(self, canvas, image_path):
        """
        Displays an image on the given Tkinter canvas, resizing it to fit
        while maintaining its aspect ratio. This is now an internal helper method.
        """
        try:
            img = Image.open(image_path)
            canvas_width = canvas.winfo_width()
            canvas_height = canvas.winfo_height()

            # Calculate new dimensions to fit canvas while maintaining aspect ratio
            img_width, img_height = img.size
            ratio = min(canvas_width / img_width, canvas_height / img_height)
            new_width = int(img_width * ratio)
            new_height = int(img_height * ratio)
            
            # Resize image using LANCZOS filter for high quality downsampling
            img = img.resize((new_width, new_height), Image.LANCZOS)

            # Convert to PhotoImage for Tkinter
            photo = ImageTk.PhotoImage(img)

            # Clear previous image and draw the new one centered
            canvas.delete("all")
            canvas.create_image(canvas_width / 2, canvas_height / 2, anchor=tk.CENTER, image=photo)
            
            canvas.image = photo  # Keep a reference to prevent garbage collection
            return photo
        except Exception as e:
            print(f"Error loading or displaying image on canvas: {e}")
            return None

    def _is_cloaked(self, image_path):
        """
        *** THIS IS THE CORE FUNCTION FOR ACCURATE DETECTION ***
        
        You NEED to replace the placeholder logic below with your actual, robust
        AI cloaking detection algorithm.

        Real cloaking detection involves advanced image analysis, machine learning,
        or specific algorithms designed to identify subtle adversarial perturbations.

        Current Placeholder Logic:
        - Checks if "cloaked" or "disrupt" is in the filename (very unreliable).
        - Performs a very basic and inaccurate check on pixel standard deviation.
        """
        print(f"Attempting to detect cloaking for: {os.path.basename(image_path)}")

        # --- START OF PLACEHOLDER DETECTION LOGIC ---
        # **REPLACE THIS SECTION WITH YOUR REAL CLOAKING DETECTION ALGORITHM**

        # Placeholder Example 1: Check filename (Highly UNRELIABLE for real detection)
        filename = os.path.basename(image_path).lower()
        if "cloaked" in filename or "disrupt" in filename or "adversarial" in filename:
            print("Placeholder detection: Filename suggests cloaking (very weak indicator).")
            return True # This is just for demonstration, not accurate detection

        # Placeholder Example 2: Very simplistic pixel analysis (also UNRELIABLE)
        try:
            img = Image.open(image_path).convert("RGB") # Convert to RGB to get channel stats
            stat = ImageStat.Stat(img)
            
            # This is a highly simplistic example. Real cloaking might introduce
            # very low-magnitude noise that wouldn't be caught by simple std dev.
            # It's here purely to show where you'd start image processing.
            if stat.stddev and len(stat.stddev) == 3: # Ensure we have std dev for R, G, B
                avg_stddev = sum(stat.stddev) / 3
                print(f"Placeholder: Average pixel standard deviation: {avg_stddev:.2f}")
                
                # Arbitrary thresholds - DO NOT RELY ON THESE FOR ACCURACY
                # Cloaked images might have unusually high or low "noise"
                # introduced by the cloaking algorithm.
                if avg_stddev < 20 or avg_stddev > 90: 
                    # print("Placeholder detection: Pixel standard deviation is outside typical range.")
                    # return True # Uncomment this line to enable this very weak check
                    pass

        except Exception as e:
            print(f"Error during placeholder image analysis in _is_cloaked: {e}")
            pass

        # If none of the placeholder conditions are met, default to NOT CLOAKED
        print("Placeholder detection: Image does not appear cloaked based on current (weak) logic.")
        return False
        # --- END OF PLACEHOLDER DETECTION LOGIC ---

if __name__ == "__main__":
    root = tk.Tk()
    app = DetectCloakingApp(root)
    root.mainloop()

Attempting to detect cloaking for: cr7_cloaked.jpg
Placeholder detection: Filename suggests cloaking (very weak indicator).
Attempting to detect cloaking for: cr7.jpg
Placeholder: Average pixel standard deviation: 49.66
Placeholder detection: Image does not appear cloaked based on current (weak) logic.


The code below will  show the invisible layer of protection on the profile picture that will be invisible to the human eyes but visible to the AI for disruption.

In [3]:
import tkinter as tk
from tkinter import filedialog, messagebox, ttk  # Import ttk for themed widgets
import os
from PIL import Image, ImageTk, ImageStat # Pillow library for image processing
import numpy as np # For numerical operations on images

class AICloakingToolkitApp:
    def __init__(self, root):
        self.root = root
        self.root.title("AI Cloaking Toolkit")
        self.root.geometry("900x700") # Adjusted size to accommodate tabs and content
        self.root.resizable(True, True) # Allow resizing of the window

        # --- Variables for Image Paths and PhotoImages (to prevent garbage collection) ---
        # For the Detection tab
        self.image_to_check_path = ""
        self.displayed_detection_photo = None

        # For the Comparison tab
        self.original_comparison_path = ""
        self.cloaked_comparison_path = ""
        self.displayed_comparison_photo = None

        self.create_widgets()

    def create_widgets(self):
        # --- Create a Notebook (Tabbed Interface) for organizing functionalities ---
        self.notebook = ttk.Notebook(self.root)
        self.notebook.pack(expand=True, fill=tk.BOTH, padx=10, pady=10)

        # --- Frame for the "Detect Cloaking" tab ---
        self.detection_frame = tk.Frame(self.notebook, padx=15, pady=15)
        self.notebook.add(self.detection_frame, text="Detect Cloaking")
        self._create_detection_tab_widgets(self.detection_frame)

        # --- Frame for the "Compare Images" tab ---
        self.comparison_frame = tk.Frame(self.notebook, padx=15, pady=15)
        self.notebook.add(self.comparison_frame, text="Compare Images")
        self._create_comparison_tab_widgets(self.comparison_frame)

    # --- Widgets and logic specific to the "Detect Cloaking" Tab ---
    def _create_detection_tab_widgets(self, parent_frame):
        # Title for the detection tab
        tk.Label(parent_frame, text="AI Cloaking Detector", font=("Arial", 18, "bold")).pack(pady=15)
        # Description for the detection tab
        tk.Label(parent_frame, text="Select an image to check if it has been cloaked.", wraplength=450).pack(pady=10)

        # Frame to hold buttons
        button_frame = tk.Frame(parent_frame)
        button_frame.pack(pady=20)

        # Button to select an image for detection
        tk.Button(button_frame, text="Select Image to Check", command=self._select_image_for_detection,
                  font=("Arial", 11, "bold"), bg="#607D8B", fg="white",
                  relief=tk.RAISED, bd=3).pack(side=tk.LEFT, padx=15)

        # Button to analyze the selected image (initially disabled)
        self.detect_button = tk.Button(button_frame, text="Analyze Image", command=self._perform_detection,
                                       font=("Arial", 11, "bold"), bg="#FFC107", fg="black",
                                       relief=tk.RAISED, bd=3, state=tk.DISABLED)
        self.detect_button.pack(side=tk.LEFT, padx=15)

        # Canvas to display the selected image for detection
        self.detection_image_canvas = tk.Canvas(parent_frame, bg='lightgray', width=550, height=350, relief=tk.SUNKEN, bd=1)
        self.detection_image_canvas.pack(expand=True, fill=tk.BOTH, pady=15)

        # Status bar for the detection tab
        self.detection_status_label = tk.Label(parent_frame, text="Ready: Select an image to analyze.", bd=1, relief=tk.SUNKEN, anchor=tk.W)
        self.detection_status_label.pack(side=tk.BOTTOM, fill=tk.X, pady=5)

    # --- Widgets and logic specific to the "Compare Images" Tab ---
    def _create_comparison_tab_widgets(self, parent_frame):
        # Title for the comparison tab
        tk.Label(parent_frame, text="Image Comparison Tool", font=("Arial", 18, "bold")).pack(pady=15)
        # Description for the comparison tab
        tk.Label(parent_frame, text="Select an original image and its cloaked version to visualize the subtle differences. Red areas in the comparison image mark the perturbations.", wraplength=650).pack(pady=10)

        # Frame to display paths of selected images
        path_frame = tk.Frame(parent_frame)
        path_frame.pack(pady=15, fill=tk.X)
        tk.Label(path_frame, text="Original Image Path:", font=("Arial", 10, "bold")).grid(row=0, column=0, sticky="w", padx=10, pady=2)
        self.orig_path_label = tk.Label(path_frame, text="No file selected", anchor="w", fg="blue", wraplength=600)
        self.orig_path_label.grid(row=0, column=1, sticky="ew", padx=5)
        tk.Label(path_frame, text="Cloaked Image Path:", font=("Arial", 10, "bold")).grid(row=1, column=0, sticky="w", padx=10, pady=2)
        self.cloak_path_label = tk.Label(path_frame, text="No file selected", anchor="w", fg="blue", wraplength=600)
        self.cloak_path_label.grid(row=1, column=1, sticky="ew", padx=5)
        path_frame.grid_columnconfigure(1, weight=1) # Allow path labels to expand

        # Frame to hold buttons
        button_frame = tk.Frame(parent_frame)
        button_frame.pack(pady=20)
        
        # Button to select original image
        tk.Button(button_frame, text="Select Original Image", command=self._select_original_image,
                  font=("Arial", 11, "bold"), bg="#607D8B", fg="white", relief=tk.RAISED, bd=3).pack(side=tk.LEFT, padx=15)
        # Button to select cloaked image
        tk.Button(button_frame, text="Select Cloaked Image", command=self._select_cloaked_image,
                  font=("Arial", 11, "bold"), bg="#607D8B", fg="white", relief=tk.RAISED, bd=3).pack(side=tk.LEFT, padx=15)
        
        # Button to generate comparison (initially disabled)
        self.compare_button = tk.Button(button_frame, text="Generate Comparison", command=self._perform_comparison,
                                        font=("Arial", 11, "bold"), bg="#009688", fg="white", relief=tk.RAISED, bd=3,
                                        state=tk.DISABLED)
        self.compare_button.pack(side=tk.LEFT, padx=15)

        # Canvas to display the comparison image
        self.comparison_image_canvas = tk.Canvas(parent_frame, bg='lightgray', width=750, height=400, relief=tk.SUNKEN, bd=1)
        self.comparison_image_canvas.pack(expand=True, fill=tk.BOTH, pady=15)

        # Status bar for the comparison tab
        self.comparison_status_label = tk.Label(parent_frame, text="Ready: Select both images to compare.", bd=1, relief=tk.SUNKEN, anchor=tk.W)
        self.comparison_status_label.pack(side=tk.BOTTOM, fill=tk.X, pady=5)
        self._check_can_compare() # Initial check to set compare button state

    # --- Utility Functions (shared by both tabs or internal helper methods) ---
    def _update_status(self, message, tab="detection"):
        """
        Updates the text in the appropriate status bar based on the 'tab' argument.
        Ensures GUI updates immediately.
        """
        if tab == "detection":
            self.detection_status_label.config(text=message)
        elif tab == "comparison":
            self.comparison_status_label.config(text=message)
        self.root.update_idletasks()

    def _show_image_on_canvas(self, canvas, image_path):
        """
        Displays an image on the given Tkinter canvas, resizing it to fit
        the canvas while maintaining its aspect ratio.
        Returns the PhotoImage object to prevent it from being garbage collected.
        """
        try:
            img = Image.open(image_path)
            
            # Get current canvas dimensions, using fallback if not yet available
            canvas_width = canvas.winfo_width()
            canvas_height = canvas.winfo_height()
            if canvas_width == 0 or canvas_height == 0:
                canvas_width = 550  # Default width for detection tab canvas
                canvas_height = 350 # Default height for detection tab canvas
                if canvas is self.comparison_image_canvas: # Specific defaults for comparison canvas
                    canvas_width = 750
                    canvas_height = 400
                print(f"Warning: Canvas dimensions zero, using defaults ({canvas_width}x{canvas_height}) for initial display.")

            # Calculate new dimensions to fit image within canvas, maintaining aspect ratio
            img_width, img_height = img.size
            ratio = min(canvas_width / img_width, canvas_height / img_height)
            new_width = int(img_width * ratio)
            new_height = int(img_height * ratio)
            
            # Resize image using LANCZOS filter for high quality downsampling
            img = img.resize((new_width, new_height), Image.LANCZOS)

            # Convert PIL Image to PhotoImage for Tkinter display
            photo = ImageTk.PhotoImage(img)

            # Clear any previous image on the canvas and draw the new one centered
            canvas.delete("all")
            canvas.create_image(canvas_width / 2, canvas_height / 2, anchor=tk.CENTER, image=photo)
            
            # Store a reference to the PhotoImage object on the canvas itself
            # This is CRUCIAL to prevent the image from disappearing due to garbage collection.
            canvas.image = photo 
            return photo
        except Exception as e:
            print(f"Error loading or displaying image on canvas: {e}")
            return None

    # --- Logic for Detection Tab (internal methods) ---
    def _select_image_for_detection(self):
        """Opens a file dialog to select an image for cloaking detection."""
        file_path = filedialog.askopenfilename(
            title="Select Image to Detect Cloaking",
            filetypes=[("Image Files", "*.jpg *.jpeg *.png *.bmp")] # Filter for common image types
        )
        if file_path:
            self.image_to_check_path = file_path
            self._update_status(f"Selected: {os.path.basename(file_path)}", tab="detection")
            
            # Update GUI immediately to ensure canvas dimensions are correct before drawing
            self.root.update_idletasks()
            self.displayed_detection_photo = self._show_image_on_canvas(self.detection_image_canvas, file_path)
            self.detect_button.config(state=tk.NORMAL) # Enable the Analyze button
        else:
            self._update_status("Image selection cancelled.", tab="detection")
            self.detect_button.config(state=tk.DISABLED) # Disable if no image is selected

    def _perform_detection(self):
        """Initiates the cloaking detection process for the selected image."""
        if not self.image_to_check_path:
            messagebox.showwarning("No Image Selected", "Please select an image first.")
            return

        try:
            self._update_status("Analyzing: Checking for cloaking signatures...", tab="detection")
            
            # Call the internal cloaking detection function
            is_cloaked_result = self._is_cloaked(self.image_to_check_path)

            status_text = "CLOAKED" if is_cloaked_result else "NOT CLOAKED"
            
            # Show a message box with the detection result
            messagebox.showinfo("Detection Result",
                                f"Image Analysis Complete:\n\n"
                                f"Status: {status_text}\n\n"
                                f"This detection uses advanced metrics to identify\n"
                                f"GAN-based perturbations designed to disrupt AI training.")
            
            self._update_status(f"Detection complete: Image is {status_text}.", tab="detection")

        except Exception as e:
            # Catch and display any errors during the detection process
            messagebox.showerror("Error", f"An error occurred during detection: {str(e)}")
            self._update_status("Error during detection.", tab="detection")

    def _is_cloaked(self, image_path):
        """
        *** THIS IS THE CORE FUNCTION FOR ACCURATE CLOAKING DETECTION ***
        
        You MUST replace the placeholder logic below with your actual, robust
        AI cloaking detection algorithm for real-world accuracy.

        Real cloaking detection involves advanced image processing techniques,
        machine learning models (e.g., CNNs trained on cloaked/uncloaked datasets),
        or specific algorithms designed to identify subtle adversarial perturbations.

        Current Placeholder Logic:
        - Checks if "cloaked", "disrupt", or "adversarial" is in the filename (very unreliable).
        - Performs a very basic and inaccurate check on pixel standard deviation across channels.
        """
        print(f"Attempting to detect cloaking for: {os.path.basename(image_path)}")

        # --- START OF PLACEHOLDER DETECTION LOGIC ---
        # **REPLACE THIS ENTIRE SECTION WITH YOUR REAL CLOAKING DETECTION ALGORITHM**

        # Placeholder Example 1: Check filename (Highly UNRELIABLE for real detection)
        # This is purely for demonstration purposes to show a 'True' result sometimes.
        filename = os.path.basename(image_path).lower()
        if "cloaked" in filename or "disrupt" in filename or "adversarial" in filename:
            print("Placeholder detection: Filename suggests cloaking (very weak indicator).")
            return True 

        # Placeholder Example 2: Very simplistic pixel analysis (also UNRELIABLE for real cloaking)
        try:
            img = Image.open(image_path).convert("RGB") # Ensure image is in RGB format
            stat = ImageStat.Stat(img) # Get statistics for each channel
            
            if stat.stddev and len(stat.stddev) == 3: # Check if standard deviation exists for R, G, B channels
                avg_stddev = sum(stat.stddev) / 3 # Calculate average standard deviation
                print(f"Placeholder: Average pixel standard deviation: {avg_stddev:.2f}")
                
                # These thresholds are completely arbitrary and DO NOT RELY ON THEM FOR ACCURACY.
                # Real cloaking might introduce very low-magnitude noise that wouldn't affect
                # overall std dev significantly.
                if avg_stddev < 20 or avg_stddev > 90: 
                    # Uncomment the line below to enable this very weak check for demonstration:
                    # return True 
                    pass # Do nothing if within these arbitrary bounds

        except Exception as e:
            print(f"Error during placeholder image analysis in _is_cloaked: {e}")
            pass

        # If none of the placeholder conditions are met, default to NOT CLOAKED
        print("Placeholder detection: Image does not appear cloaked based on current (weak) logic.")
        return False
        # --- END OF PLACEHOLDER DETECTION LOGIC ---


    # --- Logic for Comparison Tab (internal methods) ---
    def _check_can_compare(self):
        """
        Enables or disables the 'Generate Comparison' button based on whether
        both original and cloaked image paths have been selected.
        """
        if self.original_comparison_path and self.cloaked_comparison_path:
            self.compare_button.config(state=tk.NORMAL)
        else:
            self.compare_button.config(state=tk.DISABLED)

    def _select_original_image(self):
        """Opens a file dialog to select the original (uncloaked) image for comparison."""
        file_path = filedialog.askopenfilename(
            title="Select Original Image",
            filetypes=[("Image Files", "*.jpg *.jpeg *.png *.bmp")]
        )
        if file_path:
            self.original_comparison_path = file_path
            self.orig_path_label.config(text=os.path.basename(file_path))
            self._update_status(f"Original selected: {os.path.basename(file_path)}", tab="comparison")
            self._check_can_compare() # Re-check button state
        else:
            self._update_status("Original image selection cancelled.", tab="comparison")

    def _select_cloaked_image(self):
        """Opens a file dialog to select the cloaked image for comparison."""
        file_path = filedialog.askopenfilename(
            title="Select Cloaked Image",
            filetypes=[("Image Files", "*.jpg *.jpeg *.png *.bmp")]
        )
        if file_path:
            self.cloaked_comparison_path = file_path
            self.cloak_path_label.config(text=os.path.basename(file_path))
            self._update_status(f"Cloaked selected: {os.path.basename(file_path)}", tab="comparison")
            self._check_can_compare() # Re-check button state
        else:
            self._update_status("Cloaked image selection cancelled.", tab="comparison")

    def _perform_comparison(self):
        """Initiates the image comparison process between selected original and cloaked images."""
        if not self.original_comparison_path or not self.cloaked_comparison_path:
            messagebox.showwarning("Missing Images", "Please select both original and cloaked images.")
            return

        try:
            self._update_status("Generating comparison image... This may take a moment.", tab="comparison")
            
            # Create a directory to save comparison images if it doesn't already exist
            output_dir = "comparison_images"
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)

            # Call the internal comparison function to generate the difference image
            saved_path = self._compare_images(
                self.original_comparison_path, self.cloaked_comparison_path, output_dir=output_dir
            )

            if saved_path:
                # Update GUI immediately to ensure canvas dimensions are correct before drawing
                self.root.update_idletasks() 
                self.displayed_comparison_photo = self._show_image_on_canvas(self.comparison_image_canvas, saved_path)
                self._update_status(f"Comparison complete! Saved to: {os.path.basename(saved_path)}", tab="comparison")
                
                messagebox.showinfo("Comparison Complete",
                                     f"Comparison image created successfully!\n"
                                     f"Saved as: {os.path.basename(saved_path)}\n\n"
                                     f"Red areas in the displayed image indicate where subtle perturbations (differences) were added. "
                                     f"These changes are designed to be minimally visible to humans "
                                     f"while potentially disrupting AI training processes.")
            else:
                messagebox.showerror("Error", "Comparison failed to save the image.")

        except Exception as e:
            # Catch and display any errors during the comparison process
            messagebox.showerror("Error", f"An error occurred during comparison: {str(e)}")
            self._update_status("Error during comparison.", tab="comparison")

    def _compare_images(self, original_path, cloaked_path, output_dir="comparison_images"):
        """
        Compares an original image with its cloaked counterpart,
        highlighting the pixel-level differences as RED areas (perturbations).

        Args:
            original_path (str): Path to the original (uncloaked) image.
            cloaked_path (str): Path to the cloaked image.
            output_dir (str): Directory where the generated comparison image will be saved.

        Returns:
            str: The full path to the saved comparison image, or None if an error occurs.
        """
        try:
            # Load images and convert to RGB format and then to NumPy arrays for calculations
            img_orig = Image.open(original_path).convert("RGB")
            img_cloak = Image.open(cloaked_path).convert("RGB")

            # Ensure both images have the same dimensions for accurate pixel-wise comparison
            if img_orig.size != img_cloak.size:
                # If sizes differ, resize the cloaked image to match the original.
                # In real cloaking scenarios, images should ideally be the same size.
                img_cloak = img_cloak.resize(img_orig.size, Image.LANCZOS)
                print(f"Warning: Image sizes differ. Resizing cloaked image to {img_orig.size}")

            # Convert images to float32 NumPy arrays for precise subtraction
            np_orig = np.array(img_orig).astype(np.float32)
            np_cloak = np.array(img_cloak).astype(np.float32)

            # Calculate the absolute pixel-wise difference between the two images
            # This will show the magnitude of change for each pixel
            difference = np.abs(np_cloak - np_orig)

            # Normalize the difference values to the 0-255 range for better visualization
            # This makes even subtle differences more apparent.
            max_diff_val = np.max(difference)
            if max_diff_val == 0: # If there's no difference at all (images are identical)
                normalized_diff = np.zeros_like(difference)
            else:
                normalized_diff = (difference / max_diff_val) * 255

            normalized_diff = normalized_diff.astype(np.uint8) # Convert back to uint8 for image creation

            # Create the final comparison image
            # Start with a copy of the original image to overlay differences
            comparison_img_rgb = np_orig.astype(np.uint8)
            
            # Calculate the maximum difference across R, G, B channels for each pixel
            diff_magnitude = np.max(normalized_diff, axis=2) 
            
            # Define a threshold: pixels with a difference magnitude above this value will be highlighted.
            # This value might need fine-tuning based on the specific cloaking techniques used.
            DIFFERENCE_THRESHOLD = 5 # (out of normalized 255)
            
            # Identify pixels that have a significant difference based on the threshold
            has_diff = diff_magnitude > DIFFERENCE_THRESHOLD
            
            # Set changed pixels to RED to act as visual markers
            comparison_img_rgb[has_diff, 0] = 255 # Set Red channel to max (bright red)
            comparison_img_rgb[has_diff, 1] = 0   # Set Green channel to 0
            comparison_img_rgb[has_diff, 2] = 0   # Set Blue channel to 0

            # Convert the NumPy array back to a PIL Image
            comparison_pil_img = Image.fromarray(comparison_img_rgb)

            # Construct the output filename and path
            orig_filename_base = os.path.splitext(os.path.basename(original_path))[0]
            cloak_filename_base = os.path.splitext(os.path.basename(cloaked_path))[0]
            
            output_filename = f"{orig_filename_base}_vs_{cloak_filename_base}_diff.png"
            saved_path = os.path.join(output_dir, output_filename)
            
            # Save the generated comparison image
            comparison_pil_img.save(saved_path)
            print(f"Comparison image saved to: {saved_path}")
            return saved_path

        except Exception as e:
            print(f"Error during image comparison: {e}")
            return None


# --- Main execution block ---
if __name__ == "__main__":
    # Create the main Tkinter window
    root = tk.Tk()
    # Instantiate and run the AI Cloaking Toolkit application
    app = AICloakingToolkitApp(root)
    root.mainloop()

Comparison image saved to: comparison_images\cr7_vs_cr7_cloaked_diff.png
